In [14]:
import os
os.chdir("../")

In [51]:
%pwd

'c:\\Users\\teck1\\OneDrive\\Documents\\GitHub\\ai'

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    checkpoint_filepath_str: str

In [53]:
from DetectAnalytics.constants import *
from DetectAnalytics.utils.common import read_yaml, create_directories

In [64]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.preparing_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            checkpoint_filepath_str=config.checkpoint_model_filepath
        )
        
        return prepare_callback_config

In [55]:
import os
import urllib.request as request
import zipfile
import tensorflow as tf
import time

In [65]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_filepath_str,
            save_best_only=True
        )
    
    @property
    def _create_earlystopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor = 'val_accuracy',
            patience = 10,
            restore_best_weights = True
        )
    
    @property
    def _create_reducelr_callbacks(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            factor = 0.6,
            patience = 5,
            min_lr = 0.00001,
            verbose = 1
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_earlystopping_callbacks,
            self._create_reducelr_callbacks
        ]

In [66]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-12-17 22:09:29,083: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-17 22:09:29,087: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-17 22:09:29,089: INFO: common: created directory at: artifacts]
[2023-12-17 22:09:29,090: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-17 22:09:29,091: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [67]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-12-17-22-10-05'